In [33]:
# we will be create graph
# ! pip install -r requirements.txt 
# ! pip install langchain_groq
! pip install langchain


Defaulting to user installation because normal site-packages is not writeable


In [34]:
def fun1(input_string):
    return input_string + "I am data scientist"

def fun2(input_string):
    return input_string + "I am gen ai developer"

In [35]:
from langgraph.graph import StateGraph
import os
from langchain_groq import ChatGroq
from typing import TypedDict,Optional
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel
from dotenv import load_dotenv

In [36]:


# load .env
load_dotenv()

True

In [37]:

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [64]:
model = ChatGroq(model = "qwen/qwen3-32b")


#typedict for langraph
class Disasterstate(TypedDict,total = False):
    summary:Optional[str]
    date: Optional[str]
    location: Optional[str]
    country : Optional[str]
    state: Optional[str]

# define pydantic class
class Disasterentity(BaseModel):
    date: Optional[str]
    location: Optional[str]
    country : Optional[str]
    state: Optional[str]


llm_with_parser = model.with_structured_output(Disasterstate)

In [75]:
summary_template = """
You are an expert disaster report summarizer.

Task:
Summarize the given news article into a clear and concise paragraph 
focusing only on natural calamities that occurred in 2023.

Rules:
- Ignore historical events, anniversaries, organizational updates, or policy statements.
- Include only disaster-related facts from 2023.
- If multiple years are mentioned, summarize only 2023 events.

Summary must include (if available):
- Date, day, or month of the calamity
- Location (village, district, state, country, etc.)
- Duration (how long it lasted)
- Number of lives lost
- Infrastructure damaged (roads, bridges, houses, power, schools, etc.)
- Losses for farmers (crops, livestock, farmland, financial impact)
- Total estimated loss

Output Format:
- One well-structured paragraph
- Do not add extra information unrelated to 2023 calamities

Example Input:
"A massive flood hit Bihar on August 10, lasting for 5 days. Over 200 people died,
and thousands of houses and bridges were destroyed. Farmers reported crop damage
worth 500 crore rupees. The government announced 10,000 rupees compensation per
affected family and set up relief camps."

Example Output:
On August 10, a devastating flood struck Bihar and lasted for five days, claiming
over 200 lives. The disaster caused widespread destruction, washing away thousands
of houses and bridges, while farmers suffered agricultural losses worth 500 crore
rupees. 
"""

In [ ]:
def summarise_article(state:Disasterstate) -> Disasterstate:
    
    article = state.get("article",summary_template)

    prompt = ChatPromptTemplate.from_messages([
    ("system",),
    ("human","Article:\n{input_text}")])

    chain = prompt|llm_with_parser

    summary = chain.invoke({"input_text":article})

    return summary

    

In [66]:

def extract_data(state:Disasterstate):

    article = state.get("article","")

    prompt = ChatPromptTemplate.from_messages([
                            ("system","you are smart AI extracter you have to get date, loction, country,state from disaster article"),
                            ("human", "Article:\n{input_text}")])
                            
    chain = prompt|llm_with_parser

    return chain.invoke({"input_text":article})
    

In [67]:
# create graph using langraph

workflow = StateGraph(Disasterstate)

workflow.add_node("summarizer",summarise_article)

workflow.add_node("extract_entities",extract_data)

workflow.add_edge("summarizer","extract_entities")

workflow.set_entry_point("summarizer")

workflow.set_finish_point("extract_entities")


In [68]:
app = workflow.compile()

In [74]:
# from IPython.display import Image, display

# display(Image(app.get_graph().draw_mermaid_png(max_retries=5, retry_delay=2.0)))



In [70]:
article_text = """in december, intense rainfall affected several districts in tamil nadu, including tuticorin and 
tirunelveli. the release of 7,000 cusecs of water into the thamirabarani river was reported on an unspecified 
date, prompting evacuation advisories. on december 1 and 2, four districts, including tuticorin, tirunelveli, 
tenkasi, and kanyakumari, experienced heavy rainfall as per imd predictions. the extreme rainfall caused damage 
to crops raised over 2.64 lakh hectares in eight districts, resulting in a compensation of ₹250 crore. the 
government also announced a relief package worth ₹1,000 crore. approximately 17,000 heads of cattle and one lakh 
chickens were reported dead in tuticorin and tirunelveli districts."""

result = app.invoke({"article": article_text})

print(result)

{'summary': 'A major earthquake with a magnitude of 7.2 struck Los Angeles, California, causing significant damage and casualties.', 'date': 'March 15, 2023', 'location': 'Los Angeles', 'country': 'USA', 'state': 'California'}


In [73]:
result["date"]

'March 15, 2023'